In [2]:
# Installer les packages nécessaires (plotly, ipywidgets)
# Cette cellule peut être cachée dans la version finale avec :tags: [remove-input]
%pip install seaborn pyvista --quiet

Note: you may need to restart the kernel to use updated packages.


# 🎯 Traitement et analyse statistique des données de forage

## 📘 Analyse du Dégroupement sur un Gisement Synthétique Spatialement Corrélé

Dans cet atelier, nous explorerons les effets de l'échantillonnage et du dégroupement sur un gisement spatialement corrélé suivant une distribution marginale log-normale. Nous analyserons comment la taille des cellules de dégroupement influence les estimations de la moyenne et de la variance des échantillons. Les participants apprendront à visualiser et interpréter les résultats à travers des graphiques interactifs.

### 🖼️ Descriptions des 4 figures

1. **Carte du champ log-normal avec échantillons :**  
   Cette figure montre la distribution spatiale du champ log-normal simulé, avec les emplacements des échantillons aléatoires (en noir) et ceux ciblant une zone à forte valeur (hotspot, en rouge).

2. **Histogrammes et fonctions de répartition cumulée (CDF) :**  
   On compare ici la distribution des valeurs du champ complet avec celles des échantillons, en incluant une version pondérée si le dégroupement est activé, pour visualiser les biais d’échantillonnage.

3. **Effet de la taille de cellule sur la moyenne pondérée :**  
   Cette courbe montre comment la moyenne des échantillons varie en fonction de la taille des cellules utilisées pour le dégroupement, comparée à la moyenne globale du champ.

4. **Effet de la taille de cellule sur la variance pondérée :**  
   De manière similaire, cette figure illustre l’impact de la taille de cellule sur la variance estimée des échantillons, en la comparant à la variance réelle du champ.

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, IntSlider, FloatSlider, fixed, ToggleButton

def spherical_cov(h, range_):
    c = np.zeros_like(h)
    mask = h < range_
    hr = h[mask] / range_
    c[mask] = 1 - 1.5 * hr + 0.5 * hr**3
    return c

def fftma_spherical(n, range_, sigma=1.0, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    N = 2 * n
    x = np.arange(N)
    y = np.arange(N)
    X, Y = np.meshgrid(x, y)
    dist_x = np.minimum(X, N - X)
    dist_y = np.minimum(Y, N - Y)
    h = np.sqrt(dist_x**2 + dist_y**2)
    
    cov = spherical_cov(h, range_)
    cov = cov / cov.max()
    
    fft_cov = np.fft.fft2(cov)
    white_noise = np.random.normal(0, 1, (N, N))
    fft_noise = np.fft.fft2(white_noise)
    
    fft_field = np.sqrt(np.abs(fft_cov)) * fft_noise
    field = np.fft.ifft2(fft_field).real
    
    field = field[:n, :n]
    field = (field - np.mean(field)) / np.std(field)
    field = field * sigma
    return field

def generate_lognormal_field(n, range_, sigma_gauss, median=1.0, seed=None):
    gauss_field = fftma_spherical(n, range_, sigma=sigma_gauss, seed=seed)
    mu = np.log(median) - 0.5 * sigma_gauss**2
    lognormal_field = np.exp(mu + gauss_field)
    return gauss_field, lognormal_field

def compute_cell_declustering_weights(samples, n_cells):
    cells = {}
    weights = np.zeros(len(samples))
    for i, (x, y) in enumerate(samples):
        cx = int(x / n_cells)
        cy = int(y / n_cells)
        key = (cx, cy)
        if key not in cells:
            cells[key] = []
        cells[key].append(i)
    
    for indices in cells.values():
        w = 1.0 / len(indices)
        for idx in indices:
            weights[idx] = w
    return weights

def find_hotspot_location(field, hotspot_size):
    n = field.shape[0]
    max_pos = np.unravel_index(np.argmax(field), field.shape)
    x0 = max(0, max_pos[0] - hotspot_size // 2)
    y0 = max(0, max_pos[1] - hotspot_size // 2)
    x1 = min(n, x0 + hotspot_size)
    y1 = min(n, y0 + hotspot_size)
    return x0, y0, x1, y1

def interactive_sampling_fixed_hotspot(
    n, range_, sigma_gauss, median, n_samples_all,
    n_samples_hotspot, hotspot_size, declust_cells,
    show_declustering, x0y0x1y1, seed
):
    gauss_field, lognormal_field = generate_lognormal_field(n, range_, sigma_gauss, median, seed=seed)
    x0, y0, x1, y1 = x0y0x1y1

    indices_all = np.array([(i,j) for i in range(n) for j in range(n)])
    rng = np.random.default_rng(seed)
    
    sampled_indices_all = rng.choice(len(indices_all), size=n_samples_all, replace=False)
    samples_all = indices_all[sampled_indices_all]
    samples_all_values = lognormal_field[samples_all[:,0], samples_all[:,1]]

    indices_hotspot = np.array([(i,j) for i in range(x0, x1) for j in range(y0, y1)])
    if len(indices_hotspot) == 0:
        samples_hotspot = np.empty((0,2), dtype=int)
        samples_hotspot_values = np.array([])
    else:
        n_samples_hotspot = min(n_samples_hotspot, len(indices_hotspot))
        sampled_indices_hotspot = rng.choice(len(indices_hotspot), size=n_samples_hotspot, replace=False)
        samples_hotspot = indices_hotspot[sampled_indices_hotspot]
        samples_hotspot_values = lognormal_field[samples_hotspot[:,0], samples_hotspot[:,1]]
    
    combined_samples = np.vstack([samples_all, samples_hotspot])
    combined_values = np.concatenate([samples_all_values, samples_hotspot_values])

    weights = compute_cell_declustering_weights(combined_samples, declust_cells)
    weights /= weights.sum()
    
    # Nouvelle figure 2x2
    fig, axs = plt.subplots(2, 2, figsize=(10, 8))

    # === Carte du champ ===
    ax = axs[0, 0]
    im = ax.imshow(lognormal_field, cmap='jet', origin='lower')
    ax.scatter(samples_all[:,1], samples_all[:,0], facecolors='none', edgecolors='black', s=40, label='Échantillons aléatoires')
    ax.scatter(samples_hotspot[:,1], samples_hotspot[:,0], facecolors='none', edgecolors='red', s=60, label='Échantillons hotspot')
    ax.set_title("Gisement avec échantillons")
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.legend()
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04, label='Valeur log-normal')

    # === Histogrammes et CDF ===
    ax = axs[0, 1]
    bins = np.linspace(0, np.percentile(lognormal_field, 99.5), 30)
    
    sns.histplot(lognormal_field.flatten(), bins=bins, stat="density",
                 color='gray', label='Champ complet', ax=ax, alpha=0.3)
    sns.histplot(combined_values, bins=bins, stat="density",
                 color='blue', label='Échantillons', ax=ax, alpha=0.4)

    if show_declustering:
        ax.hist(combined_values, bins=bins, weights=weights, density=True,
                color='green', label='Dégroupés (pondérés)', alpha=0.5,
                edgecolor='black', linewidth=0.5)

    ax.set_xlabel("Valeur")
    ax.set_ylabel("Densité")
    ax.set_title("Histogrammes")

    ax2 = ax.twinx()
    sorted_all = np.sort(lognormal_field.flatten())
    sorted_non = np.sort(combined_values)
    sorted_idx = np.argsort(combined_values)
    cdf_all = np.linspace(0, 1, len(sorted_all))
    cdf_non = np.linspace(0, 1, len(sorted_non))
    cdf_w = np.cumsum(weights[sorted_idx])

    ax2.plot(sorted_all, cdf_all, color='black', lw=2, linestyle='--', label="CDF champ complet")
    ax2.plot(sorted_non, cdf_non, color='blue', lw=2, linestyle='-', label="CDF échantillons")
    if show_declustering:
        ax2.plot(sorted_non, cdf_w, color='green', lw=2, linestyle='-', label="CDF dégroupée")

    ax2.set_ylabel("Fonction de répartition")
    ax2.set_ylim(0, 1)

    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='lower right')

    # === Nouvelle figure : moyenne pondérée vs taille de cellule ===
    ax = axs[1, 0]
    cell_sizes = np.arange(1, 100)
    means = []
    for c in cell_sizes:
        w = compute_cell_declustering_weights(combined_samples, c)
        w /= w.sum()
        means.append(np.sum(w * combined_values))
    ax.plot(cell_sizes, means, '-o', color='purple')
    ax.axvline(x=declust_cells, color='red', linestyle='--', label=f'Taille sélectionnée = {declust_cells}')
    ax.axhline(y=np.mean(combined_values), color='blue', linestyle='--', label='Moyenne Non pondéré')
    ax.axhline(y=np.mean(lognormal_field), color='black', linestyle=':', label='Moyenne globale')
    ax.set_xlabel("Taille de cellule de dégroupement")
    ax.set_ylabel("Moyenne pondérée d'échantillon")
    ax.set_title("Effet de la taille de cellule sur la moyenne d'échantillon")
    ax.grid(True)
    ax.legend(loc='upper left', fontsize=10)


    # === Quatrième graphique : variance pondérée vs taille de cellule ===
    ax = axs[1, 1]
    variances = []
    for c in cell_sizes:
        w = compute_cell_declustering_weights(combined_samples, c)
        w /= w.sum()
        mean_c = np.sum(w * combined_values)
        var_c = np.sum(w * (combined_values - mean_c)**2)
        variances.append(var_c)
    ax.plot(cell_sizes, variances, '-o', color='darkgreen')
    ax.axvline(x=declust_cells, color='red', linestyle='--', label=f'Taille sélectionnée = {declust_cells}')
    ax.axhline(y=np.var(combined_values), color='blue', linestyle='--', label='Variance Non pondéré')
    ax.axhline(y=np.var(lognormal_field), color='black', linestyle=':', label='Variance globale')
    ax.set_xlabel("Taille de cellule de dégroupement")
    ax.set_ylabel("Variance pondérée d'échantillon")
    ax.set_title("Effet de la taille de cellule sur la variance d'échantillon")
    ax.grid(True)
    ax.legend(loc='upper left', fontsize=10)

    plt.tight_layout()
    plt.show()


# === Initialisation champ et localisation hotspot (une seule fois) ===
_n = 100
_range_ = 15
_sigma = 0.6
_median = 2.0
_seed = 42
_hotspot_size = 25

_, init_field = generate_lognormal_field(_n, _range_, _sigma, _median, seed=_seed)
x0y0x1y1 = fixed(find_hotspot_location(init_field, _hotspot_size))

# === Interface interactive ===
interact(
    interactive_sampling_fixed_hotspot,
    n=fixed(_n),
    range_=IntSlider(min=5, max=50, step=1, value=_range_, description='Portée sphérique'),
    sigma_gauss=FloatSlider(min=0.1, max=1.5, step=0.1, value=_sigma, description='Variance'),
    median=FloatSlider(min=0.5, max=5.0, step=0.1, value=_median, description='Moyenne'),
    n_samples_all=IntSlider(min=10, max=500, step=10, value=100, description='N aléatoire'),
    n_samples_hotspot=IntSlider(min=0, max=200, step=10, value=50, description='N hotspot'),
    hotspot_size=fixed(_hotspot_size),
    declust_cells=IntSlider(min=1, max=100, step=1, value=10, description='Cell declustering'),
    show_declustering=ToggleButton(value=False, description='Afficher dégroupement'),
    x0y0x1y1=x0y0x1y1,
    seed=fixed(_seed)
)

interactive(children=(IntSlider(value=15, description='Portée sphérique', max=50, min=5), FloatSlider(value=0.…

<function __main__.interactive_sampling_fixed_hotspot(n, range_, sigma_gauss, median, n_samples_all, n_samples_hotspot, hotspot_size, declust_cells, show_declustering, x0y0x1y1, seed)>

Déviation

In [7]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

def d2r(deg):
    return np.deg2rad(deg)

# Méthode tangentielle équilibrée
def balanced_tangential_segment(MD, inc1, azi1, inc2, azi2):
    inc1, azi1 = d2r(inc1), d2r(azi1)
    inc2, azi2 = d2r(inc2), d2r(azi2)
    dx = (MD/2) * (np.sin(inc1) * np.sin(azi1) + np.sin(inc2) * np.sin(azi2))
    dy = (MD/2) * (np.sin(inc1) * np.cos(azi1) + np.sin(inc2) * np.cos(azi2))
    dz = -(MD/2) * (np.cos(inc1) + np.cos(inc2))  # Z positif vers le haut
    return dx, dy, dz

def interpolate_path(measurements, targets):
    measurements = sorted(measurements, key=lambda x: x[0])
    positions = [(0.0, 0.0, 0.0)]
    mids = []
    depths = [m[0] for m in measurements]

    for i in range(1, len(measurements)):
        md = depths[i] - depths[i-1]
        inc1, azi1 = measurements[i-1][1], measurements[i-1][2]
        inc2, azi2 = measurements[i][1], measurements[i][2]
        dx, dy, dz = balanced_tangential_segment(md, inc1, azi1, inc2, azi2)
        last = positions[-1]
        next_pos = (last[0]+dx, last[1]+dy, last[2]+dz)
        positions.append(next_pos)
        mids.append((depths[i-1] + md/2, last[0]+dx/2, last[1]+dy/2, last[2]+dz/2))

    interp_XYZ = []
    for t in targets:
        for i in range(1, len(depths)):
            if depths[i-1] <= t <= depths[i]:
                f = (t - depths[i-1]) / (depths[i] - depths[i-1])
                x1, y1, z1 = positions[i-1]
                x2, y2, z2 = positions[i]
                xt = x1 + f * (x2 - x1)
                yt = y1 + f * (y2 - y1)
                zt = z1 + f * (z2 - z1)
                interp_XYZ.append((t, xt, yt, zt))
                break
    return positions, mids, interp_XYZ

def plotly_3d_path(path, mids, interp, measurements):
    path = np.array(path)
    fig = go.Figure()

    # Ligne de trajectoire
    fig.add_trace(go.Scatter3d(
        x=path[:,0], y=path[:,1], z=path[:,2],
        mode='lines',
        line=dict(color='blue', width=5),
        name='Trajectoire'
    ))

    # Points d'angles (stations)
    station_coords = np.array([path[i] for i in range(len(measurements))])
    fig.add_trace(go.Scatter3d(
        x=station_coords[:,0], y=station_coords[:,1], z=station_coords[:,2],
        mode='markers+text',
        marker=dict(size=6, color='orange'),
        text=[f"Station {i} ({m[0]} m)" for i, m in enumerate(measurements)],
        textposition='top center',
        name='Stations (mesures)'
    ))

    # Points "mids" (point milieu par segment)
    if mids:
        mids_np = np.array(mids)
        fig.add_trace(go.Scatter3d(
            x=mids_np[:,1], y=mids_np[:,2], z=mids_np[:,3],
            mode='markers+text',
            marker=dict(size=5, color='green'),
            text=[f"{d:.1f} m" for d in mids_np[:,0]],
            textposition='top center',
            name='Points milieux'
        ))

    # Points composites (interpolés)
    if interp:
        interp = np.array(interp)
        fig.add_trace(go.Scatter3d(
            x=interp[:,1], y=interp[:,2], z=interp[:,3],
            mode='markers+text',
            marker=dict(size=5, color='red'),
            text=[f"{d:.1f} m" for d in interp[:,0]],
            textposition='bottom center',
            name='Composites (interpolation)'
        ))

    fig.update_layout(
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m, ↑)',
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=1)
        ),
        title='Trajectoire 3D corrigée (Z vers le haut)',
        margin=dict(l=0, r=0, b=0, t=40),
        legend=dict(x=0, y=1)
    )
    fig.show()

# Widgets
num_points_w = widgets.BoundedIntText(value=3, min=2, max=20, description="Stations:")
md_list_w = widgets.Text(value="0, 80, 180", description="MD (m):")
inc_list_w = widgets.Text(value="90, 80, 70", description="Inclinaison (°):")
azi_list_w = widgets.Text(value="60, 55, 45", description="Azimut (°):")
targets_w = widgets.Text(value="180, 183, 186", description="Cibles MD (m):")
button = widgets.Button(description="Tracer", button_style='primary')
out = widgets.Output()

def on_button_clicked(b):
    with out:
        clear_output()
        try:
            md_list = list(map(float, md_list_w.value.strip().split(',')))
            inc_list = list(map(float, inc_list_w.value.strip().split(',')))
            azi_list = list(map(float, azi_list_w.value.strip().split(',')))
            targets = list(map(float, targets_w.value.strip().split(',')))

            n = num_points_w.value
            if not (len(md_list) == len(inc_list) == len(azi_list) == n):
                print("⚠️ Erreur : les longueurs des listes ne correspondent pas au nombre de stations.")
                return

            measurements = list(zip(md_list, inc_list, azi_list))
            path, mids, interp = interpolate_path(measurements, targets)
            plotly_3d_path(path, mids, interp, measurements)
        except Exception as e:
            print(f"❌ Erreur : {str(e)}")

button.on_click(on_button_clicked)

# Explication des conventions
info_text = Markdown("""
### ℹ️ Conventions de forage
- **Azimut (°)** : mesuré à partir du **nord**, **dans le plan horizontal**, dans le **sens horaire**.
- **Inclinaison (°)** : mesurée à partir de l’**horizontale vers le bas** (0° = horizontale, 90° = verticale descendante).
- **Z positif vers le haut** (forage descend en Z négatif).
- **Points affichés** :
    - 🔶 **Stations** : positions où les angles sont mesurés.
    - 🟩 **Points milieux** : segments interpolés (méthode tangentielle équilibrée).
    - 🔴 **Composites** : positions de MD cibles interpolées.
""")

display(info_text, num_points_w, md_list_w, inc_list_w, azi_list_w, targets_w, button, out)



### ℹ️ Conventions de forage
- **Azimut (°)** : mesuré à partir du **nord**, **dans le plan horizontal**, dans le **sens horaire**.
- **Inclinaison (°)** : mesurée à partir de l’**horizontale vers le bas** (0° = horizontale, 90° = verticale descendante).
- **Z positif vers le haut** (forage descend en Z négatif).
- **Points affichés** :
    - 🔶 **Stations** : positions où les angles sont mesurés.
    - 🟩 **Points milieux** : segments interpolés (méthode tangentielle équilibrée).
    - 🔴 **Composites** : positions de MD cibles interpolées.


BoundedIntText(value=3, description='Stations:', max=20, min=2)

Text(value='0, 80, 180', description='MD (m):')

Text(value='90, 80, 70', description='Inclinaison (°):')

Text(value='60, 55, 45', description='Azimut (°):')

Text(value='180, 183, 186', description='Cibles MD (m):')

Button(button_style='primary', description='Tracer', style=ButtonStyle())

Output()

In [1]:
# Exemple réaliste avec 100 stations
n_points = 100
md_list = np.linspace(0, 1000, n_points)  # Mesures le long du trou
inc_list = np.linspace(0, 80, n_points)   # Inclinaison de 0° à 80°, progressivement
azi_list = 45 + 5 * np.sin(np.linspace(0, 3*np.pi, n_points))  # Azimut autour de 45° (léger sinus)

measurements = list(zip(md_list, inc_list, azi_list))
targets = np.linspace(0, 1000, 25)  # Interpolation à 25 cibles réparties

# Calcul du chemin, des mids, et des points interpolés
path, mids, interp = interpolate_path(measurements, targets)

# Tracé 3D
plotly_3d_path(path, mids, interp, measurements)


NameError: name 'np' is not defined